# Functions as First class Citizens

In [2]:
x = 100

def foo(y):
    return x+y

z = foo(307)
print(x,z,foo)
def bar(x):
    x=1000
    return foo(308)
w=bar(349)
print(x,w)

def apply_n_times(f,n,x):
    out = x
    for i in range(n):
        out=f(out)

    return out


def double(x):
    return x*2

def outer(func):
    def inner():
        print("Hi")
    return inner

@outer
def sayHi():
    print("Hello")
    
sayHi()




100 407 <function foo at 0x105d1ff70>
100 408
Hi


# Decorators and Closures

In [15]:
def decorate(func):
    def inner():
        print("Inner Function")
        return 2*func() # Know little this level of python
    return inner


"""
Decorators first execute their function, then lazily execute the function they are called with and finally calculate the result of decorated function
"""
@decorate
def target():
   print("Target Function")
   return 2

#print(decorate(target()))
print("Target:",target())


Inner Function
Target Function
Target: 4


In [27]:
"""
Calculate double of a function 
"""

def double_func(func):
    def inner(x): # x is passed by the target function to the decorated function
        print(x) 
        return 2*func(x)
    return inner
    

@double_func
def half_of_x(x):
    x=x//2
    print(x)
    return x

print(half_of_x(3)) # Apparently 2 is being passed



3
1
2


In [28]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def foo1():
    print("foo1 Function")
    
@register
def foo2():
    print("foo2 Function")


def foo3():
    print("foo3 Function")
    


running register(<function foo1 at 0x110ecd080>)
running register(<function foo2 at 0x110ece8e0>)


In [29]:
print("Invoking Register",registry)

Invoking Register [<function foo1 at 0x110ecd080>, <function foo2 at 0x110ece8e0>]


In [30]:
print(decorate(target))

<function decorate.<locals>.inner at 0x110ece840>


In [31]:
print(register(foo1))

running register(<function foo1 at 0x110ecd080>)
<function foo1 at 0x110ecd080>


# Note that register runs (twice) before any other function in the module. When register is called, it receives the decorated function object as an argument—

for example, <function f1 at 0x100631bf8>.

In [32]:
for reg in registry:
    reg()

foo1 Function
foo2 Function
foo1 Function


In [33]:
foo1()

foo1 Function


In [34]:
registry[1]

<function __main__.foo2()>

In [35]:
registry[0]()

foo1 Function


In [36]:
for reg in registry:
    reg()

foo1 Function
foo2 Function
foo1 Function


In [40]:
import Register

In [41]:
Register.registry

[<function Register.foo1()>,
 <function Register.foo2()>,
 <function Register.foo3()>]

3
6


# Closures

In [89]:
class Averager:
    """
    Starter for closures and OOPS
    variable with _ has class scope and __ is a private attribute
    """
    def __init__(self):
        
        self._series = []
        self.__series1 = []
    
    def __call__(self,val):
        self._series.append(val)
        self.__series1.append(val)
        total = sum(self._series)/len(self._series)
        
        return total
        

In [90]:
avg = Averager()
print(avg(10))
print(avg(100))

10.0
55.0


In [93]:
print(avg._series)
# So wondering why bother with private static protected and public variables in Java



[10, 100]


In [92]:
avg.__series1

AttributeError: 'Averager' object has no attribute '__series1'

In [98]:
"""
Functional Way
Closure
"""

def make_average():
    series = []
    def calculate_average(num):
        series.append(num)
        total = sum(series)/len(series)
        return total
    return calculate_average

In [99]:
avg = make_average()
print(avg(10))

10.0


In [100]:
avg(11)

10.5

In [101]:
avg(12)

11.0

In [102]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [114]:
avg.__code__.co_freevars

('series',)

In [7]:
from dis import dis

In [104]:
dis(foo1)

  8           0 RESUME                   0

 10           2 LOAD_GLOBAL              1 (NULL + print)
             12 LOAD_CONST               1 ('foo1 Function')
             14 CALL                     1
             22 POP_TOP
             24 RETURN_CONST             0 (None)


In [106]:
dis(make_average)

              0 MAKE_CELL                1 (series)

  6           2 RESUME                   0

  7           4 BUILD_LIST               0
              6 STORE_DEREF              1 (series)

  8           8 LOAD_CLOSURE             1 (series)
             10 BUILD_TUPLE              1
             12 LOAD_CONST               1 (<code object calculate_average at 0x1079606f0, file "/var/folders/zt/1scshgnj4jxfkz9kb1005c8r0000gn/T/ipykernel_16435/1073538618.py", line 8>)
             14 MAKE_FUNCTION            8 (closure)
             16 STORE_FAST               0 (calculate_average)

 12          18 LOAD_FAST                0 (calculate_average)
             20 RETURN_VALUE

Disassembly of <code object calculate_average at 0x1079606f0, file "/var/folders/zt/1scshgnj4jxfkz9kb1005c8r0000gn/T/ipykernel_16435/1073538618.py", line 8>:
              0 COPY_FREE_VARS           1

  8           2 RESUME                   0

  9           4 LOAD_DEREF               2 (series)
              6 L

In [42]:
def generate_parenthesis(n):
    series = []
    def dfs(left,right,s):
        if left+right==2*n:
            series.append(s)
            return
            
        if left<n:
            dfs(left+1,right,s+'(')
        if right<left:
            dfs(left,right+1,s+')')
    
    dfs(0,0,"")
    return series

In [43]:
print(generate_parenthesis(4))

['(((())))', '((()()))', '((())())', '((()))()', '(()(()))', '(()()())', '(()())()', '(())(())', '(())()()', '()((()))', '()(()())', '()(())()', '()()(())', '()()()()']


In [109]:
gen = generate_parenthesis(4)

In [14]:
dis(generate_parenthesis) # Without Return

              0 MAKE_CELL                0 (n)
              2 MAKE_CELL                1 (dfs)
              4 MAKE_CELL                2 (series)

  1           6 RESUME                   0

  2           8 BUILD_LIST               0
             10 STORE_DEREF              2 (series)

  3          12 LOAD_CLOSURE             1 (dfs)
             14 LOAD_CLOSURE             0 (n)
             16 LOAD_CLOSURE             2 (series)
             18 BUILD_TUPLE              3
             20 LOAD_CONST               1 (<code object dfs at 0x10448c5b0, file "/var/folders/zt/1scshgnj4jxfkz9kb1005c8r0000gn/T/ipykernel_17178/3002107191.py", line 3>)
             22 MAKE_FUNCTION            8 (closure)
             24 STORE_DEREF              1 (dfs)

 13          26 PUSH_NULL
             28 LOAD_DEREF               1 (dfs)
             30 LOAD_CONST               2 (0)
             32 LOAD_CONST               2 (0)
             34 LOAD_CONST               3 ('')
             36 CALL       

In [6]:
%timeit generate_parenthesis(4)

4.8 µs ± 5.44 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [115]:
gen[0].__code__.co_freevars

AttributeError: 'str' object has no attribute '__code__'

In [16]:
dis(generate_parenthesis)

              0 MAKE_CELL                0 (n)
              2 MAKE_CELL                1 (dfs)
              4 MAKE_CELL                2 (series)

  1           6 RESUME                   0

  2           8 BUILD_LIST               0
             10 STORE_DEREF              2 (series)

  3          12 LOAD_CLOSURE             1 (dfs)
             14 LOAD_CLOSURE             0 (n)
             16 LOAD_CLOSURE             2 (series)
             18 BUILD_TUPLE              3
             20 LOAD_CONST               1 (<code object dfs at 0x10448c2f0, file "/var/folders/zt/1scshgnj4jxfkz9kb1005c8r0000gn/T/ipykernel_17178/4014062791.py", line 3>)
             22 MAKE_FUNCTION            8 (closure)
             24 STORE_DEREF              1 (dfs)

 13          26 PUSH_NULL
             28 LOAD_DEREF               1 (dfs)
             30 LOAD_CONST               2 (0)
             32 LOAD_CONST               2 (0)
             34 LOAD_CONST               3 ('')
             36 CALL       

# Non Locals

In [17]:
def calulate_average():
    count=0
    total = 0
    def averager(val):
        nonlocal count,total
        count+=1
        total+=val
        return total/count
    return averager

In [18]:
x= calulate_average()
x(10)
print(x(11))
print(x.__closure__[0].cell_contents)

10.5
2


In [54]:
list1 = [1,2,3,4,5]
list2 = ['a','b','c','d']
pairs = [pair for pair in zip(list1,list2)]

In [55]:
letters,numbers = zip(*pairs) #Unzip
print(letters,numbers)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


In [58]:
def doubler(f):
    def g(x):
        return 2*f(x)
    return g
def f1(x):
    return x+1
a = doubler(f1)
assert a(4) == 10

In [44]:
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args):
            start = time.perf_counter()
            _result = func(*args)
            elapsed = time.perf_counter()-start
            name = func.__name__
            args = ','.join(repr(arg) for arg in args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

        

In [64]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12804000s] snooze(0.123) -> None
[0.12651587s] snooze(0.123) -> None
[0.12610687s] snooze(0.123) -> None


In [65]:
@clock('{name}:{elapsed}s')
def snooze_again(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze_again(.123)

snooze_again:0.12312695799482754s
snooze_again:0.1238607499981299s
snooze_again:0.1280332919995999s


In [46]:
print(clock(generate_parenthesis(4)))

TypeError: 'list' object is not callable

In [1]:
import time
import functools
def clock1(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = (time.perf_counter() - t0)*100000
        name = func.__name__
        arg_lst = [repr(arg) for arg in args] 
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}') 
        return result
    return clocked

In [50]:
print(clock1(generate_parenthesis(4)))

<function clock1.<locals>.clocked at 0x1108bb600>


In [2]:
@clock1
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

In [3]:
print(factorial(45))

[0.03330001s] factorial(1) -> 1
[2.69999996s] factorial(2) -> 2
[3.32909999s] factorial(3) -> 6
[3.68329997s] factorial(4) -> 24
[4.02090000s] factorial(5) -> 120
[6.41669999s] factorial(6) -> 720
[8.43329999s] factorial(7) -> 5040
[8.80000002s] factorial(8) -> 40320
[9.07909998s] factorial(9) -> 362880
[9.40000000s] factorial(10) -> 3628800
[9.70000001s] factorial(11) -> 39916800
[10.07090000s] factorial(12) -> 479001600
[10.43749999s] factorial(13) -> 6227020800
[10.74590000s] factorial(14) -> 87178291200
[11.17500001s] factorial(15) -> 1307674368000
[11.48330002s] factorial(16) -> 20922789888000
[11.77919999s] factorial(17) -> 355687428096000
[12.10000000s] factorial(18) -> 6402373705728000
[12.40829997s] factorial(19) -> 121645100408832000
[12.70829998s] factorial(20) -> 2432902008176640000
[12.99999999s] factorial(21) -> 51090942171709440000
[13.32500001s] factorial(22) -> 1124000727777607680000
[13.62080002s] factorial(23) -> 25852016738884976640000
[13.90839998s] factorial(24) -

In [6]:
@clock1
def generate_parenthesis_with_return(n):
    series = []
    def dfs(left,right,s):
        if left+right==2*n:
            series.append(s)
            return
            
        if left<n:
            dfs(left+1,right,s+'(')
        if right<left:
            dfs(left,right+1,s+')')
    
    dfs(0,0,"")
    return series

In [ ]:
print(generate_parenthesis_with_return(10))

In [65]:
@clock1
def generate_parenthesis_without_return(n):
    series = []
    def dfs(left,right,s):
        if left+right==2*n:
            series.append(s)
        
            
        if left<n:
            dfs(left+1,right,s+'(')
        if right<left:
            dfs(left,right+1,s+')')
    
    dfs(0,0,"")
    return series

In [66]:
print(generate_parenthesis_without_return(5))

[3.20419999s] generate_parenthesis_without_return(5) -> ['((((()))))', '(((()())))', '(((())()))', '(((()))())', '(((())))()', '((()(())))', '((()()()))', '((()())())', '((()()))()', '((())(()))', '((())()())', '((())())()', '((()))(())', '((()))()()', '(()((())))', '(()(()()))', '(()(())())', '(()(()))()', '(()()(()))', '(()()()())', '(()()())()', '(()())(())', '(()())()()', '(())((()))', '(())(()())', '(())(())()', '(())()(())', '(())()()()', '()(((())))', '()((()()))', '()((())())', '()((()))()', '()(()(()))', '()(()()())', '()(()())()', '()(())(())', '()(())()()', '()()((()))', '()()(()())', '()()(())()', '()()()(())', '()()()()()']
['((((()))))', '(((()())))', '(((())()))', '(((()))())', '(((())))()', '((()(())))', '((()()()))', '((()())())', '((()()))()', '((())(()))', '((())()())', '((())())()', '((()))(())', '((()))()()', '(()((())))', '(()(()()))', '(()(())())', '(()(()))()', '(()()(()))', '(()()()())', '(()()())()', '(()())(())', '(()())()()', '(())((()))', '(())(()())', '(()

In [4]:
@clock1
def generate_parenthesis_using_yield(n):
    def dfs(left,right,s):
        if left+right==2*n:
            yield s
        if left < n:
            dfs(left+1,right,s+'(')
        if right < left:
            dfs(left,right+1,s+')')
    return list(dfs(0,0,""))

In [5]:
print(generate_parenthesis_using_yield(3))

[0.38749999s] generate_parenthesis_using_yield(3) -> []
[]


In [ ]:
from __future__ import annotations

def tokenise(text: str)->list[str]:
    return text.upper().split()
